# PIK3CA Mutation Trans Effect on Acetylproteomics

This notebook analyzes the trans effect of PIK3CA mutation on interacting and other proteins Acetylproteomics, in Endometrial cancer (The Colon and Ovarian Datasets don't have acetylproteomic data.

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.algorithms as al

en = cptac.Endometrial()
co = cptac.Colon()

### Specify Gene

In [2]:
gene = "PIK3CA"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
#omics = "transcriptomics"
#omics = "phosphoproteomics"
omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Acetylproteomics

### Generate interacting protein list

In [6]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
PIK3R2
TNS1
RPS6KB1
KRAS
PDGFRA
PIK3R3
AKT3
PIK3CA
ERBB3
IGF1R
ERBB2
EGFR
GNAQ
KIT
MRAS
IRS1
CTNNB1
NRAS
PTEN
IRS2
AKT2
CDC42
ESR1
HRAS
PIK3R1
AKT1
ATR
LCK
NOTCH1
ADAP1
DGKZ
ARHGEF1
GSN
YWHAH
IRS4
UBTF


## Endometrial

### Test for significant comparisons in any of interacting proteins

In [7]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Endometrial", True, all_significant_comparisons)


PIK3R2 did not match any columns in acetylproteomics dataframe. PIK3R2_acetylproteomics column inserted, but filled with NaN.
RPS6KB1 did not match any columns in acetylproteomics dataframe. RPS6KB1_acetylproteomics column inserted, but filled with NaN.
KRAS did not match any columns in acetylproteomics dataframe. KRAS_acetylproteomics column inserted, but filled with NaN.
PDGFRA did not match any columns in acetylproteomics dataframe. PDGFRA_acetylproteomics column inserted, but filled with NaN.
PIK3R3 did not match any columns in acetylproteomics dataframe. PIK3R3_acetylproteomics column inserted, but filled with NaN.
AKT3 did not match any columns in acetylproteomics dataframe. AKT3_acetylproteomics column inserted, but filled with NaN.
PIK3CA did not match any columns in acetylproteomics dataframe. PIK3CA_acetylproteomics column inserted, but filled with NaN.
ERBB3 did not match any columns in acetylproteomics dataframe. ERBB3_acetylproteomics column inserted, but filled with NaN.


## Colon

Acetylproteomic data does not exist for the Colon dataset

## Ovarian

Acetylproteomic data does not exist for the Ovarian dataset

# All Proteins: Phosphoproteomics

## Endometrial

In [8]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = en.get_proteomics()
    all_proteins = proteomics.columns
    all_proteins = list(all_proteins)

    all_proteins_no_dash = []
    for ap in all_proteins:
        if '-' not in ap:
            all_proteins_no_dash.append(ap)
            
    #all_proteins = all_proteins[:500]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Endometrial", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  PIK3CA
Doing t-test comparisons

No significant comparisons.


## Colon

Colon Dataset doesn't contain acetylproteomic data

## Ovarian

Ovarian Dataset doesn't contain acetylproteomic data

### Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

No Significant Comparisons!


### Write significant comparisons (if any) to shared CSV file

In [10]:
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)